# Demultiplexing, aligning and calling genotypes for RADseq data using fastq-multx and ipyrad

This notebook is for single-read RADseq data with a file for each index.  The input files are unmodified files recieved from the University of Oregon genomics core (GC3F) and a barcode file.  In this format, the data must be demultiplexed using fastq-multx and then aligned and called using ipyrad.  This file should be easily modified to handle paired end data in a similar format.  If your barcoding indexes have not been seperated for your read file (you have a single file with your read and barcode info), you can probably start directly in ipyrad and demultiplex in ipyrad as well.

This process uses a bash script for fastq-multx and a bash script for ipyrad.  Parameters for ipyrad are kept in a seperate parameter file that is specified in the ipyrad bash script.  This parameter file allows modifications that allow you to specify input data and data types but also modify the alignment and genotype calling quality control parameters or specify a reference genome for alignment.

This notebook is designed for a slurm based hpcc (specifically monsoon.hpc.nau.edu).  It requires a mamba environment named "iprenv" with ipyrad installed.  Here we use mambaforge/22.9.0.  You may need to change the version used in the ipyrad bash script if you are using a different version of mambaforge.

**Input files**: 
* R1.fastq.gz I1.fastq.gz I2.fastq.gz  Where R1 is a fastq of single-end reads and I1 and I2 are fastqs of index 1 and 2 respectively.  Manually load this into your /scratch/usr/rad/ipyrad folder.
* sep_barcodes.txt  A two column file with no header where the first column is your sample ID and the second is index1 and index2 seperated by a "-"  ex. 1RC     GTACGTTC-CCATCACA.  Manually load this into your /scratch/usr/rad/ipyrad folder.
* params.txt  An ipyrad parameter file.  Examine and modify this file to your needs.

**Bash scripts**
* multx.sh  Bash script for running fastq-multx
* ipyrad.sh  Bash script for alignment and genotype calling in ipyrad.

**Ouput files**

Output files can be found in rad_outfiles.  The parameter file is currently set to give all the possible outputs from ipyrad which include nexus, phylip, tremix, and vcf.



**Set values for your directory, set input file name:**

In [3]:
mydir = "your/directory/name"

In [4]:
cd $mydir

/scratch/mw955/mive1


**Get bash scripts and parameter files**

In [4]:
!git clone https://github.com/thalictroides/RAD_demult_align_genotype

Cloning into 'RAD_demult_align_genotype'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 17 (delta 5), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (17/17), 6.05 KiB | 2.02 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [7]:
cd RAD_demult_align_genotype

/scratch/mw955/mive1/RAD_demult_align_genotype


**Manually add your R1.fastq.gz, I1.fastq.gz, I2.fastq.gz, and sepbarcodes.txt to this directory**

In [8]:
ls

I1.fastq.gz  README.md     ipyrad.sh  sep_barcodes.txt
I2.fastq.gz  demult/       multx.sh   slurm-5960908.out
R1.fastq.gz  fastq-multx/  param.txt


## Run fastq-multx

**get fastq-multx**

In [12]:
!git clone https://github.com/brwnj/fastq-multx

Cloning into 'fastq-multx'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 99 (delta 11), reused 15 (delta 4), pack-reused 72
Receiving objects: 100% (99/99), 117.53 KiB | 610.00 KiB/s, done.
Resolving deltas: 100% (38/38), done.


In [13]:
cd fastq-multx

/scratch/mw955/mive1/RAD_demult_align_genotype/fastq-multx


In [15]:
!make

g++ -O3 -I. fastq-multx.cpp fastq-lib.cpp -o fastq-multx


In [16]:
cd ..

/scratch/mw955/mive1/RAD_demult_align_genotype


**make folder for demultiplexed reads**

In [21]:
mkdir demult

In [22]:
!sbatch multx.sh

Submitted batch job 5960908


## run ipyrad

##  ipyrad scripts
I've chosen to split my ipyrad scripts into two seperate scripts to handle the configuration limitations of Monsoon.  The first script does step 1-3.  Step 3 in ipyrad is clustering/mapping which takes large amounts of memory but not nearly as much time, so I've increased memory requirements while decreasing the amount of parallel jobs of my script.  The next script is steps 4-7.  Step 5 is genotype calling which takes more time but not nearly as much memory as clustering so my script here has decreased memory and increased parallelization.  I've included a single ipyrad script incase your hpcc configurations are different.

Both scripts call the same parameter file.  The parameters for this analysis are all the same as the defaults except that I have specified that this analysis is ddrad in order for the clustering algorithm to work properly.  I also reduced the stringency setting for the for adapter filtering so that the program doesn't exclued reads that don't include adapter sequences since these were already removed in the demultiplexing step.  I have still specified my restriction overhang despite this not being used with the current settings because the program throws an error without a restriction overhand when ddrad is specified for data type.

In [9]:
!sbatch ipyrad1.sh

Submitted batch job 5961193


In [2]:
!sbatch ipyrad2.sh

 1_get_matrix.txt             clumpK4_abgwas.R      part.nex
 2_contigs_to_probes.txt      clump_abgwas.R        part.nex.log
 5snpfiltering_SPCR.ipynb     clumpbash.sh          pca_radseq.R
 BSG_vcftools.ipynb           data.tnt              qmptut/
 IQTree_MFP.sh                filter/               rdabash.sh
 IQTree_MFP.sh.save           filtered_indv.log     simplejob.sh
 MAFFT.sh                     freq.log              snps.log
 R/                           fstruct.sh           'squeue -t R | grep mw955'
 RAD_demult_align_genotype/   lazyjob.sh            test/
 RDA_monsoon.R                mive_ipyrad.ipynb     tnt.sh
 abgwas_for.sh                mive_vcftools.ipynb   tools/
 clump.R*                     ondemand/
 clump.sh                     paml/
